# DR VAKU® Swadesi Pulse Oximeter - Analysis

https://www.amazon.in/DR-Oximeter-Fingertip-Monitor-Approved/dp/B08D3KXF9Y/ref=cm_cr_arp_d_product_top?ie=UTF8&th=1

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import spacy
import string
from wordcloud import WordCloud, STOPWORDS

%matplotlib inline

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
reviews = pd.read_csv('OxiReviews.csv')

In [ ]:
reviews.head()

In [ ]:
reviews.info()

# Rating Graph

In [ ]:
import seaborn as sns
count = reviews['rating'].value_counts()
sns.barplot(x=count.index, y = count, color = '#a2c816')

In [ ]:
reviews['rating'].value_counts()

In [ ]:
reviews = reviews.dropna(axis=0)

In [ ]:
reviews.isnull().sum()

In [ ]:
body = reviews['body']
rv = " ".join(body)
rv

In [ ]:
#To remove punctuation marks
no_punc_text = rv.translate(str.maketrans('', '', string.punctuation))

In [ ]:
no_punc_text

In [ ]:
#To Create tokens
from nltk.tokenize import word_tokenize
text_tokens = word_tokenize(no_punc_text)
print(text_tokens[0:50])

In [ ]:
len(text_tokens)

In [ ]:
#Remove Stopwords by NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
my_stop_words = stopwords.words('english')
li = ['the','The', 'in','I', 'This','its','it','It', 'dont', 'a', 'He', 'Im', 'as']
for i in li:
    my_stop_words.append(i)
no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
print(no_stop_tokens[:40])

In [ ]:
#Normalization
lower_words = [x.lower() for x in no_stop_tokens]
print(lower_words[0:25])

In [ ]:
nlp = spacy.load('en_core_web_sm') 

In [ ]:
doc = nlp(' '.join(no_stop_tokens))
print(doc[0:40])

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas[0:25])

In [ ]:
#Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lemmas)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
import seaborn as sns
dic = vectorizer.vocabulary_
data = pd.DataFrame(dic.items(), columns=['Date', 'DateValue'])
data0=data.sort_values('DateValue',ascending = False)[:10]
plt.figure(figsize = (15, 8))
sns.barplot(x='Date', y="DateValue", data = data0, color='#a2c816')


In [ ]:
data0.to_csv(r'wordsFinalFreq.csv', index=False)
data0.head(10)

In [ ]:
#Create Wordcloud
from wordcloud import WordCloud, STOPWORDS

def plot_cloud(wordcloud):
    plt.figure(figsize=(40, 30))
    plt.imshow(wordcloud) 
    plt.axis("off");

In [ ]:
stopwords = STOPWORDS
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = STOPWORDS).generate(rv)
plot_cloud(wordcloud)

In [ ]:
wordcloud.to_file("wordCLFinal.png")

In [ ]:
df = []
for i in reviews['rating']:
    if i<=2:
        i='Negative'
    elif i == 3:
        i='Neutral'
    elif i >=4:
        i='Positive'
    df.append(i)
df
reviews['Classify'] = pd.DataFrame(df)

In [ ]:
reviews.head(1000)

In [ ]:
pct_Classify = reviews['Classify'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pct_Classify

#  Vader Sentiment Analysis

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
def classify_compound(text, threshold=0.33):
     
    # initialize VADER
    sid = SentimentIntensityAnalyzer()
     
    # Calling the polarity_scores method on sid and passing in the text
    # outputs a dictionary with negative, neutral, positive, and compound scores for the input text
    scores = sid.polarity_scores(text)
     
    # get compound score
    score = scores['compound']
     
    # translate the score into the correcponding input according to the threshold
    if score <= -threshold: return 'Negative'
    elif score >= threshold: return 'Positive'
    else: return 'Neutral'

In [ ]:
reviews['compound_sentiment'] = reviews.body.apply(lambda text: classify_compound(text))

In [ ]:
reviews.head(10)

In [ ]:
pct_cs = reviews['compound_sentiment'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pct_cs

In [ ]:
plt.hist(reviews['compound_sentiment'],color='#a2c816')

In [ ]:
label = [ 'Positive', 'Neutral', 'Negative']
plt.pie(reviews['compound_sentiment'].value_counts(), labels=label)
plt.show() 

# TextBlob

In [ ]:
from textblob import TextBlob

In [ ]:
pol = lambda x : TextBlob(x).sentiment.polarity
sub = lambda x : TextBlob(x).sentiment.subjectivity

In [ ]:
reviews['Polarity'] = reviews['body'].apply(pol)
reviews['Subjectivity'] = reviews['body'].apply(sub)

In [ ]:
reviews.head()

In [ ]:
#Polarity lies between [-1,1]
def getAnalysis(score):
      if score < 0:
        return 'Negative'
      elif score == 0:
        return 'Neutral'
      else:
        return 'Positive'

In [ ]:
reviews['TextBlob'] = reviews['Polarity'].apply(getAnalysis)

In [ ]:
reviews.head()

In [ ]:
pct_tb = reviews['TextBlob'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pct_tb

In [ ]:
sns.histplot(reviews['TextBlob'], color='#a2c816')

In [ ]:
plt.pie(reviews['TextBlob'].value_counts(), labels=reviews['TextBlob'].unique())
plt.show()

# We have used three classifier here is the comparison of them

In [ ]:
pct_Classify #Through Ratings

In [ ]:
pct_cs #Vedar Classifier

In [ ]:
pct_tb #TextBlob Classifier

In [ ]:
import re
def text_cleaner (text):
    clean_data= re.sub('[^a-zA-Z]', " ", text) 
    clean_data= clean_data.lower()
    clean_data = clean_data.split()
    clean_data= ' '.join(clean_data)
    return clean_data

In [ ]:
reviews.dropna(subset = ["Classify"], inplace=True, axis=0)

In [ ]:
reviews.isnull().sum()

In [ ]:

reviews["Clean_review"]=reviews["body"].apply(lambda x: text_cleaner(str(x)))
reviews["sentiment"] = reviews["Classify"].map({'Positive': 1, 'Negative':0, 'Neutral':0})
reviews.head()

In [ ]:
reviews["sentiment"].value_counts()

# Modeling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)

In [ ]:
x = cv.fit_transform(reviews["Clean_review"]).toarray()

In [ ]:
y = reviews["sentiment"].values

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.metrics import roc_curve, auc

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=10)

In [ ]:
from sklearn.naive_bayes import BernoulliNB , GaussianNB, MultinomialNB

In [ ]:
bnb = BernoulliNB()
gnb= GaussianNB()
mnb= MultinomialNB()

# BernoulliNB

In [ ]:
bnb.fit(X_train, y_train)
y_pred = bnb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# GaussianNB

In [ ]:
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# MultinomialNB

In [ ]:
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg = LogisticRegression()

In [ ]:
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
sv = SVC()

In [ ]:
sv.fit(X_train, y_train)
y_pred = sv.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

In [ ]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier

In [ ]:
rf= RandomForestClassifier(n_estimators=300 , random_state=50)
ad= AdaBoostClassifier(n_estimators=300 , random_state=50)
gb = GradientBoostingClassifier(n_estimators=300 , random_state=5)
etc= ExtraTreesClassifier(n_estimators=300 , random_state=5)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {'n_estimators': [100,150,200,300,500],
#               'random_state': [10,20,30,40,50,60]}
# grid_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, refit = True, verbose = 3)
# grid_ad = GridSearchCV(AdaBoostClassifier(), param_grid=param_grid, refit = True, verbose = 3)
# grid_gb = GridSearchCV(GradientBoostingClassifier(), param_grid=param_grid, refit = True, verbose = 3)
# grid_etc = GridSearchCV(ExtraTreesClassifier(), param_grid=param_grid, refit = True, verbose = 3)

In [ ]:
# grid_rf.fit(X_train, y_train)

In [ ]:
#grid_rf.best_params_

# RandomForestClassifier

In [ ]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# AdaBoostClassifier

In [ ]:
ad.fit(X_train, y_train)
y_pred = ad.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# GradientBoostingClassifier

In [ ]:
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# ExtraTreesClassifier

In [ ]:
etc.fit(X_train, y_train)
y_pred = etc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# DecisionTreeClassifier 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

# VotingClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
vote_hard = VotingClassifier(estimators=[('lr', lg), ('bnb', bnb), ('ad', ad), ('mnb', mnb)], voting='hard')

In [ ]:
vote_hard.fit(X_train, y_train)

In [ ]:
vote_hard.fit(X_train, y_train)
y_pred = vote_hard.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

In [ ]:
vote_soft = VotingClassifier(estimators=[('lr', lg), ('bnb', bnb), ('ad', ad), ('mnb', mnb)], voting='soft')

In [ ]:
vote_soft.fit(X_train, y_train)
y_pred = vote_soft.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, color='#a2c816')
print(auc(fpr, tpr))

In [ ]:
text = 'Not Good'
clean_data= text_cleaner(text)
clean_data
cv1=cv.transform([clean_data])

In [ ]:
bnb.predict(cv1)

In [ ]:
ad.predict(cv1)

In [ ]:
mnb.predict(cv1)

In [ ]:
lg.predict(cv1)

In [ ]:
vote_soft.predict(cv1)

In [ ]:
# import pickle

In [ ]:
# file = open("F:ExcelR/Project 88/votingClassifier.pkl","wb")
# pickle.dump(vote_soft , file)

In [ ]:
# file2 = open("F:ExcelR/Project 88/vector.pkl","wb")
# pickle.dump(cv , file2)